In [18]:
import numpy as np  # Import the NumPy library for numerical operations.
import tkinter as tk  # Import the Tkinter library for creating the GUI.
from tensorflow.keras.datasets import mnist  # Import the MNIST dataset from Keras.
from tensorflow.keras.models import Sequential, load_model  # Import the Sequential model and load_model function.
from tensorflow.keras.layers import Dense, Flatten  # Import the Dense and Flatten layers for the neural network.
from tensorflow.keras.utils import to_categorical  # Import the function for one-hot encoding labels.
from PIL import Image, ImageDraw  # Import the Pillow library for image processing and drawing.



In [19]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()  # Load the training and testing data.

# Normalize the data
x_train = x_train.astype('float32') / 255  # Convert training data to float32 and normalize it to [0, 1].
x_test = x_test.astype('float32') / 255  # Convert testing data to float32 and normalize it to [0, 1].

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)  # Convert training labels to one-hot encoding.
y_test = to_categorical(y_test, num_classes=10)  # Convert testing labels to one-hot encoding.



In [20]:
# Create the model
model = Sequential([  # Initialize a sequential model.
    Flatten(input_shape=(28, 28)),  # Flatten the input images from 28x28 to a 1D array.
    Dense(128, activation='relu'),  # Add a dense layer with 128 neurons and ReLU activation function.
    Dense(10, activation='softmax')  # Add a dense output layer with 10 neurons and softmax activation for multi-class classification.
])

# Compile the model
model.compile(optimizer='adam',  # Use the Adam optimizer.
              loss='categorical_crossentropy',  # Use categorical crossentropy loss for multi-class classification.
              metrics=['accuracy'])  # Track accuracy during training.

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32)  # Train the model for 5 epochs with a batch size of 32.

# Save the model
model.save('my_model.keras')  # Save the trained model to a file.

# Test the model's accuracy
test_loss, test_accuracy = model.evaluate(x_test, y_test)  # Evaluate the model on the test data.
print(f'Test accuracy: {test_accuracy:.4f}')  # Print the test accuracy with 4 decimal places.


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8737 - loss: 0.4369
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 976us/step - accuracy: 0.9620 - loss: 0.1261
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 937us/step - accuracy: 0.9767 - loss: 0.0797
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 961us/step - accuracy: 0.9828 - loss: 0.0569
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 933us/step - accuracy: 0.9868 - loss: 0.0434
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - accuracy: 0.9746 - loss: 0.0846
Test accuracy: 0.9771


In [22]:
# Create the application window
class DrawApp:
    def __init__(self, master):  # Initialize the DrawApp class.
        self.master = master  # Store the main window.
        master.title("Digit Prediction Application")  # Set the title of the window.

        self.canvas = tk.Canvas(master, width=280, height=280, bg='white')  # Create a canvas for drawing.
        self.canvas.pack()  # Pack the canvas into the window.

        self.button_predict = tk.Button(master, text="Predict", command=self.predict_digit)  # Create a button to predict the digit.
        self.button_predict.pack()  # Pack the predict button into the window.

        self.button_clear = tk.Button(master, text="Clear", command=self.clear_canvas)  # Create a button to clear the canvas.
        self.button_clear.pack()  # Pack the clear button into the window.

        self.canvas.bind("<B1-Motion>", self.paint)  # Bind mouse movement to the paint function.

        self.image = Image.new("L", (280, 280), 255)  # Create a new grayscale image with a white background.
        self.draw = ImageDraw.Draw(self.image)  # Create a drawing context for the image.

        # Load the model
        self.model = load_model('my_model.keras')  # Load the saved model.

    def paint(self, event):  # Function to draw on the canvas.
        x, y = event.x, event.y  # Get the x and y coordinates of the mouse event.
        self.canvas.create_oval(x-5, y-5, x+5, y+5, fill='black', outline='black')  # Draw a small circle on the canvas.
        self.draw.ellipse([x-5, y-5, x+5, y+5], fill='black')  # Draw a small circle on the image.

    def clear_canvas(self):  # Function to clear the canvas.
        self.canvas.delete("all")  # Delete all drawings on the canvas.
        self.image = Image.new("L", (280, 280), 255)  # Reset the image to a new white background.
        self.draw = ImageDraw.Draw(self.image)  # Create a new drawing context.
        print("Canvas cleaned.")  # Print a message to the console.

    def predict_digit(self):  # Function to predict the drawn digit.
        img = self.image.resize((28, 28)).convert("L")  # Resize the image to 28x28 and convert to grayscale.
        img_array = np.array(img) / 255.0  # Convert the image to a numpy array and normalize it.
        img_array = img_array.reshape(1, 28, 28)  # Reshape the array to match the model input shape.
        img_array = np.expand_dims(img_array, axis=-1)  # Expand dimensions to include the channel.

        # Make a prediction
        predicted_digit = np.argmax(self.model.predict(img_array))  # Predict the digit and get the index of the highest probability.
        print(f'Predicted digit: {predicted_digit}')  # Print the predicted digit.

# Run the application
root = tk.Tk()  # Create the main application window.
app = DrawApp(root)  # Instantiate the DrawApp class.
root.mainloop()  # Start the Tkinter event loop.
